In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import glob
import astropy
import astropy.units as u
from datetime import timezone
import datetime

import copy

#Path to top-level do-dem directory - edit for your system.
path_to_dodem = '/Users/jmdunca2/do-dem/'
from sys import path as sys_path
sys_path.append(path_to_dodem+'/dodem/')

import lightcurves as lc

In [ ]:
import importlib

importlib.reload(lc)

datapaths=['/Users/jmdunca2/nustar/jul-2024/21012001001/',
           '/Users/jmdunca2/nustar/jul-2024/21012005001/',
           '/Users/jmdunca2/nustar/jul-2024/21012006001/',
           '/Users/jmdunca2/nustar/jul-2024/21012007001/',
           '/Users/jmdunca2/nustar/jul-2024/21012008001/']

obsids=[dp.split('/')[-2] for dp in datapaths]
print(obsids)


timeranges=[[datetime.datetime(2024, 7, 16, 11, 3), datetime.datetime(2024, 7, 16, 12, 3)],
            [datetime.datetime(2024, 7, 16, 17, 28), datetime.datetime(2024, 7, 16, 18, 28)],
            [datetime.datetime(2024, 7, 16, 19, 3,30), datetime.datetime(2024, 7, 16, 20, 4)],
            [datetime.datetime(2024, 7, 17, 4, 40), datetime.datetime(2024, 7, 17, 5, 39)],
            [datetime.datetime(2024, 7, 17, 6, 17), datetime.datetime(2024, 7, 17, 7, 17)]
           ]

from datetime import timezone
newtimeranges=[]
for timerange in timeranges:
    newtimeranges.append([t.replace(tzinfo=timezone.utc) for t in timerange])

#print(newtimeranges)

#backcolor="xkcd:black"
#backcolor="xkcd:light sage"

axiscolor='white'
#accepted events
nustar_acc_color='xkcd:apple'
#Nustar counts per energy range
nustar_cts_color=['xkcd:sky blue', 'xkcd:cerulean', 'xkcd:periwinkle', 'xkcd:cyan']
#nustar data background
nucolor='xkcd:pine green'
labelfontsize=23
tickfontsize=17


fig, axes = plt.subplots(2, 1, figsize=(65, 5), sharex=True)

plt.subplots_adjust(hspace=0)

timeranges_ = copy.deepcopy(timeranges)
timeranges_[1] = [datetime.datetime(2024, 7, 16, 17, 28), datetime.datetime(2024, 7, 16, 17, 49)]
timeranges_[-1]= [datetime.datetime(2024, 7, 17, 6, 17), datetime.datetime(2024, 7, 17, 6, 32)]

for i in range(0,len(timeranges_)):
    tr = timeranges_[i] #[timeranges[i][1], timeranges[i+1][0]]
    axes[0].axvspan(tr[0], tr[1], color=nucolor, alpha=1)#0.25)
    axes[1].axvspan(tr[0], tr[1], color=nucolor, alpha=1)#0.25)


for ax in axes:
    ax.spines['bottom'].set_color('white')
    ax.spines['top'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.spines['right'].set_color('white')
    ax.xaxis.label.set_color('white')
    ax.yaxis.label.set_color('white')
    #ax.title.set_color('white')
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    
#acc=0
first=0
for i in range(0, len(datapaths)):

    datapath=datapaths[i]
    tr=newtimeranges[i]
    
    evtA = glob.glob(datapath+'/event_cl/*A06_cl.evt')
    evtB = glob.glob(datapath+'/event_cl/*B06_cl.evt')
    hkA  = glob.glob(datapath+'/hk/*A_fpm.hk')
    hkB  = glob.glob(datapath+'/hk/*B_fpm.hk')

    #Load in the evt file (has the list of photons)
    evtdataA, hdrA = lc.load_nufiles(evtA[0])
    # Load in the hk file (has the livetime info)
    lvdataA, lvhdrA = lc.load_nufiles(hkA[0])
    evtdataB, hdrB = lc.load_nufiles(evtB[0])
    lvdataB, lvhdrB = lc.load_nufiles(hkB[0])

    eranges=[[2.5,3.5],[3.5,6.], [6.,10.], [10.,15.]]
    labels = ['NuSTAR 2.5-3.5 keV', 'NuSTAR 3.5-6 keV', 'NuSTAR 6-10 keV', 'NuSTAR 10-15 keV']
    ind=0
    acc=0
    for erange in eranges:
        kevA = evtdataA['PI']*0.04+1.6
        erange_evtdataA = evtdataA[np.where(np.logical_and(kevA > erange[0],kevA < erange[1]))]
        kevB = evtdataB['PI']*0.04+1.6
        erange_evtdataB = evtdataB[np.where(np.logical_and(kevB > erange[0],kevB < erange[1]))]

    
        res = lc.get_a_nustar_lightcurve(erange_evtdataA, hdrA, lvdataA, lvhdrA, timebin=10, livetime_corr=True, event_stats=True)
        times_converted, countrate, lvt, counts, acc_sample, rej_sample, all_sample = res
    
        keepinds = np.nonzero(np.logical_and(times_converted > tr[0], times_converted < tr[1]))
        times_converted=times_converted[keepinds]
        countrateA=countrate[keepinds]
    
        if acc==0:
            acc_sample=acc_sample[keepinds]
            rej_sample=rej_sample[keepinds]
            evsumA=acc_sample/(acc_sample+rej_sample)*100
        
        
        res = lc.get_a_nustar_lightcurve(erange_evtdataB, hdrB, lvdataB, lvhdrB, timebin=10, livetime_corr=True, event_stats=True)
        times_converted, countrate, lvt, counts, acc_sample, rej_sample, all_sample = res
    
        keepinds = np.nonzero(np.logical_and(times_converted > tr[0], times_converted < tr[1]))
        times_converted=times_converted[keepinds]
        countrateB=countrate[keepinds]

        if acc==0:
            acc_sample=acc_sample[keepinds]
            rej_sample=rej_sample[keepinds]
        
            evsumB=acc_sample/(acc_sample+rej_sample)

            evsum = (np.array(evsumA)+np.array(evsumB))/2.
            #axes[0].plot(times_converted, evsum, label='Accepted Event %', color=nustar_acc_color)
            acc+=1
            if first==0:
                axes[0].plot(times_converted, evsum, label='Accepted Event %', color=nustar_acc_color)

            else:
                axes[0].plot(times_converted, evsum, color=nustar_acc_color)
                
            #print(acc, first)
        if first==0:    
            axes[1].plot(times_converted, (countrateA+countrateB), label=labels[ind], color=nustar_cts_color[ind]) 
        else:
            axes[1].plot(times_converted, (countrateA+countrateB), color=nustar_cts_color[ind])
            
        ind+=1
    first+=1

axes[0].set_yscale('log')
axes[1].set_yscale('log')

axes[0].set_ylabel('% Accepted', fontsize=labelfontsize)
axes[1].set_ylabel('Counts', fontsize=labelfontsize)

axes[0].set_yticks([0.1, 1, 10, 100], labels=['0.1','1','10','100'], fontsize=tickfontsize)
axes[1].set_yticks([1e2, 1e4, 1e6])
axes[1].tick_params(axis='y', which='major', labelsize=tickfontsize)

#axes[1].set_ylim([1e3, 1e7])
axes[0].set_ylim([0.007,100])
axes[0].set_xlim([timeranges[0][0], timeranges[-1][1]])

axes[0].grid(True, which='major', axis='y', color=axiscolor, linestyle=':')
axes[1].grid(True, which='major', axis='y', color=axiscolor, linestyle=':')

#fig.patch.set_facecolor(backcolor)
#axes[0].set_facecolor(backcolor)
#axes[1].set_facecolor(backcolor)
axes[1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
axes[1].xaxis.set_minor_locator(mdates.MinuteLocator(interval=10))
axes[1].tick_params(axis='x', which='major', labelsize=tickfontsize)

#Add Goes Curve (both days)
tr=[timeranges[0][0],timeranges[-1][1]]
print(tr)
#print(tr)
lc.get_goes(tr, satellite=16)
instrument='GOES'
data = lc.load_lightcurves(instrument)

ylabel = data['GOES flux label']
goestimes = data['GOES Times']
xrsbcounts = data['XRSB counts']
xrsblabel = data['XRSB label']
gts = [t.datetime for t in goestimes]

axg = axes[0].twinx()
#xrs=[x.value for x in all_xrsb]
axg.plot(gts, xrsbcounts, label='GOES '+xrsblabel, color='red')
axg.set_yscale('log')
axg.set_ylim([1e-7,3e-4])
axg.yaxis.set_ticks([1e-6, 1e-5, 1e-4], labels=['C','M','X'], color='red', fontsize=labelfontsize)
axg.grid(True, which='major', axis='y', linestyle=':', color='red')


#Add orbit boxes
losttimeranges=[[datetime.datetime(2024, 7, 16, 12, 38), datetime.datetime(2024, 7, 16, 13, 40)],
                [datetime.datetime(2024, 7, 16, 14, 14), datetime.datetime(2024, 7, 16, 15, 16)],
                [datetime.datetime(2024, 7, 16, 15, 50), datetime.datetime(2024, 7, 16, 16, 52)]]

for tr in timeranges:
    for t in tr:
        axes[0].axvline(t, color=axiscolor)
        axes[1].axvline(t, color=axiscolor)

redalpha=0.3

for tr in losttimeranges:
    axes[0].axvspan(tr[0], tr[1], color='red', alpha=redalpha)
    axes[1].axvspan(tr[0], tr[1], color='red', alpha=redalpha)
    for t in tr:
        axes[0].axvline(t, color=axiscolor)
        axes[1].axvline(t, color=axiscolor)


        
lostsuborbits=[[datetime.datetime(2024, 7, 16, 17, 49), datetime.datetime(2024, 7, 16, 18, 28)],
                [datetime.datetime(2024, 7, 17, 6, 32), datetime.datetime(2024, 7, 17, 7, 17)]]      
for tr in lostsuborbits:
    axes[0].axvspan(tr[0], tr[1], color='red', alpha=redalpha)
    axes[1].axvspan(tr[0], tr[1], color='red', alpha=redalpha)


#leg = axes[0].legend(loc='best')
#p = leg.get_window_extent()
#print(p)

leg = fig.legend(loc='outside upper right', fontsize=labelfontsize, framealpha=0.6)
for line in leg.get_lines():
    line.set_linewidth(6.0)

plt.savefig('whole_time_lc.png', transparent=True)